## 1. PULPO Problem formulation

### 1.1. PULPO supplementary material formulation
This is the formulation found in the supplementary material of the PULPO paper and includes the "Slack" variable formulation which allows the production quantities (supply) to be defined and the demand to become an intrinsic variable
$$
    \begin{align}
        & \underset{s, slack}{\text{min}}  && \sum_{h}{w_h \cdot z_h} \\
        & \text{s.t.}   && \sum_{j}(a_{i,j}\cdot s_j) = f_i + slack_i && \forall i \\
        &               && s_j^{low} \leq s_j \leq s_j^{high} && \forall j \\
        &               && 0 \leq slack_i \leq slack_i^{high} && \forall u \\
        &               && z_h = \sum_e \sum_j (q_{h,e}\cdot b_{e,j} \cdot s_j) && \forall h \\
        &               && z_h \leq z_h^{high} && \forall h \\
        &               && \sum_j b_{e,j} \cdot s_j \leq b_e^{high} && \forall e
    \end{align}
$$

#### Description from supplementary material

The underlying optimization problem solved by PULPO can be summarized as
(OP2):

This notation is similar to the matrix notation of the TCM, detailed in
the Supplementary Information S1. Here, $i$ is the set of products, $j$ the
set of processes, $e$ the set of environmental flows and $h$ the set of
impact categories. The coefficients $a$ are the entries of the
technosphere matrix, $b$ the entries of the biosphere matrix, $q$ the
characterization factors for the respective impact categories, and $f$ are
the entries of the functional unit vector. Apart from that, $s^{low}$ and
$s^{high}$ are introduced as upper and lower bounds on the scaling vector
$s$, whose entries constitute the decision variables of the problem. The
main differences to the TCM are highlighted in the equations in
**bold**. A straightforward extension of the constraints is the addition
of a lower bound ($s_j^{low}$) for the scaling vector entries, enforcing a minimum internal production volume. Most notably, a slack variable

"$slack_i$" has been introduced for the special case where the
practitioner wants to specify the total production quantities (supply)
rather than the amount available for use outside the system (demand).
This case can be modelled by specifying identical lower and upper bounds
($s_j^{low} = s_j^{high}$) on a scaling vector entry equal to the desired
amount produced. For this particular process $j$ the corresponding product
slack variable $slack_i^{high}$ must be set to a large value. Otherwise,
it takes the value 0 and regular demand specification is performed for
the product $i$ being the reference product of process $j$.

The objective function introduces a weighted sum of all the considered
indicators $𝑧_h$ with the weights $w_h$ for $h$ in the set of impact
categories. Single objective optimization can be performed by setting
the corresponding weight of the chosen impact category $h*$ to 1 and the
rest to 0.

Additional types of capacity constraints have been added to the model
via the impact bound $z_h^{high}$ and the environmental flow bound $b_e^{high}$.

## Nomenclature 
**Sets:**
|  notation           |     description                              |
|---|---|
|  $e$                 |  Set of environmental flows                 |
|  $h$                 |  Set of impact categories                   |
|  $i$                 |  Set of products                            |
|  $j$                 |  Set of processes                           |

**Variables:**
|  notation           |     description                              |
|---|---|
|  $s_ j$              |  Scaling vector                             |
|  $slack_i$           |  Slack variable for supply definition       |
| $z_ h$                |  Impacts                                    |

**Parameters:** 
|  notation           |     description                              |
|---|---|
|  $𝑎_{i,j}$             | Technosphere matrix coefficients              |
|  $b_{e,j}$             | Biosphere matrix coefficients                 |
|  $f_i$               | Final demand values                           |
|   $q_{h,e}$            | Characterization factors                      |
|  $w_h$               | Weights for a combined objective              |
|  $s_j^{low}$          | Lower bound on production capacity            |
|  $s_j^{high}$         | Upper bound on production capacity            |
|  $slack_i^{high}$         | Upper bound on slack variable                 |
|  $z_h^{high}$         | Impact bound                                  |
|  $b_e^{high}$         | Environmental flow bound                      |


### 1.2. PULPO Pyomo formulation


$$
    \begin{align}
        & \text{(OBJ)}\                 && \text{min}    && \sum_{h}{z_h \cdot w_h}    && \forall h \in INDICATOR\\
        & \text{(FINAL\_DEMAND\_CNSTR)}\ && \text{s.t.}   && \sum_{j\in PO_i}(a_{i,j}\cdot s_j) = f_i + slack_i  && \forall i \in PRODUCT \\        
        & \text{(IMPACTS\_CNSTR)}\      &&               && z_h = \sum_{i\in ECI_h} \sum_{j\in ECO_{i, h}} (c_{i,j,h}\cdot s_j) && \forall h \in INDICATOR \\        
        & \text{(INVENTORY\_CNSTR)}\    &&               && \sum_{j \in IO_g} b_{g,j} \cdot s_j = p_g && \forall g \in INV \\        
        & \text{(UPPER\_CNSTR)}\       &&               && s_j \leq s_j^{high} && \forall j \in PROCESS \\        
        & \text{(LOWER\_CNSTR)}\       &&               && s_j \geq s_j^{low} && \forall j \in PROCESS \\       
        & \text{(INV\_CNSTR)}\         &&               && p_g \leq p_g^{high} && \forall g \in INV \\        
        & \text{(IMP\_CNSTR)}\         &&               && z_h \leq z_h^{high} && \forall h \in INDICATOR \\        
        & \text{(SLACK\_CNSTR)}\       &&               && slack_j \leq 1e20 \cdot m_j && \forall j \in PRODUCT
    \end{align}
$$

**Variables**
$$
    \begin{aligned}
        &z_h = \text{impacts[h]} &&= \text{Environmental impact on indicator h evaluated with the established LCIA method}  \\        
        &s_j = \text{scaling\_vector[j]} &&= \text{Activity level of each process to meet the final demand}  \\        
        &slack_i = \text{slack[i]} &&= \text{Slack variable for supply constraints}  \\        
        &p_g = \text{inv\_vector[g]} &&= \text{Intervention flow associated with environmental impacts, official nomenclature is "g", "p" used to not mix up with the index}  \\    
    \end{aligned}
$$

**Parameters**
$$
    \begin{aligned}
        &w_h = \text{WEIGHTS[h]} &&= \text{Weights for the impact categories, typically 1 or 0}  \\        
        &a_{i,j} = \text{TECH\_MATRIX[i,j]} &&= \text{Technosphere matrix defining product flows between processes}  \\        
        &f_i = \text{FINAL\_DEMAND[i]}  &&= \text{Final demand of intermediate product flows}  \\        
        &c_{i,j,h} = \text{ENV\_COST\_MATRIX[i,j,h]} &&= \text{Environmental cost matrix describing cost flows associated with processes}  \\        
        &b_{g,j} = \text{INV\_MATRIX[g,j]} &&= \text{Intervention matrix defining elementary flows for processes}  \\        
        &s_j^{high} = \text{UPPER\_LIMIT}[j] &&= \text{Maximum production capacity of process j}  \\        
        &s_j^{low} = \text{LOWER\_LIMIT}[j] &&= \text{Minimum production capacity of process j}  \\        
        &p_g^{high} = \text{UPPER\_INV\_LIMIT}[g] &&= \text{Maximum intervention flow g}  \\        
        &z_h^{high} = \text{UPPER\_IMP\_LIMIT}[h] &&= \text{Maximum environmental impact for indicator h}  \\        
        &m_j = \text{SUPPLY}[j] &&= \text{Binary parameter specifying if supply is defined for process j}  \\    
    \end{aligned}
$$

**Sets**
$$
    \begin{aligned}
        &INDICATOR = \text{INDICATOR} &&= \text{Set of impact assessment indicators, indexed by h}  \\        
        &PRODUCT = \text{PRODUCT} &&= \text{Set of intermediate products (or technosphere exchanges), indexed by i}  \\        
        &PROCESS = \text{PROCESS} &&= \text{Set of processes, indexed by j}  \\        
        &E = \text{ENV\_COST} &&= \text{Set of environmental cost flows, indexed by e}  \\        
        &INV = \text{INV} &&= \text{Set of intervention flows, indexed by g}  \\        
        &ECI = \text{ENV\_COST\_IN[h]} &&= \text{Subset of environmental cost flows associated with indicator h}  \\        
        &ECO = \text{ENV\_COST\_OUT[i,h]} &&= \text{Subset of environmental cost flows associated with product i and indicator h}  \\        
        &IO = \text{INV\_OUT[g]} &&= \text{Set of processes associated with intervention flow g}  \\        
        &PO = \text{PROCESS\_OUT[i]} &&= \text{Set of processes associated with product i}  \\    \end{aligned}
$$


### 1.3. Differences between pyomo and supplementary material

#### Final Demand Constraint
- **pyomo notation**:
  $$\sum_{j \in PO_i} a_{i,j} \cdot s_j = f_i + slack_i, \quad \forall i \in PRODUCT$$
- **article notation**:
  $$\sum_{j} a_{i,j} \cdot s_j = f_i + slack_i, \quad \forall i$$

**Difference**: pyomo notation specifies the subset $PO_i$ of processes, while article notation applies the summation over all $j$.

---

#### Impact Constraint
- **pyomo notation**:
  $$z_h = \sum_{i \in ECI_h} \sum_{j \in ECO_{i,h}} c_{i,j,h} \cdot s_j, \quad \forall h \in INDICATOR$$
- **article notation**:
  $$z_h = \sum_{e} \sum_{j} q_{h,e} \cdot b_{e,j} \cdot s_j, \quad \forall h$$

**Difference**:
- pyomo notation uses $c_{i,j,h}$ (environmental cost matrix) and subsets $ECI_h$, $ECO_{i,h}$.
- article notation uses $q_{h,e}$ (characterization factors) and $b_{e,j}$ (biosphere matrix), summing over $e$ and $j$ without subsets.

---

#### Inventory Constraint
- **pyomo notation**:
  $$\sum_{j \in IO_g} b_{g,j} \cdot s_j = p_g, \quad \forall g \in INV$$
- **article notation**:
  $$\sum_{j} b_{e,j} \cdot s_j \leq b_e^{high}, \quad \forall e$$

**Difference**:
- pyomo notation defines $IO_g$ as a subset and introduces $p_g$ (intervention vector).
- article notation directly constrains the environmental flow $b_e^{high}$ without an explicit $p_g$.

---

#### Slack Constraint
- **pyomo notation**:
  $$slack_j \leq 1e20 \cdot m_j, \quad \forall j \in PRODUCT$$
- **article notation**:
  $$0 \leq slack_i \leq slack_i^{high}, \quad \forall i$$

**Difference**:
- pyomo notation uses a binary parameter $m_j$ (supply flag) for controlling slack.
- article notation explicitly bounds slack using $slack_i^{high}$.

---

#### Environmental Flow Bound
- **pyomo notation**:
  Does not include this constraint explicitly.
- **article notation**:
  $$\sum_{j} b_{e,j} \cdot s_j \leq b_e^{high}, \quad \forall e$$

**Difference**: article notation includes a specific constraint for bounding environmental flows, which is absent in pyomo notation.

---

#### Nomenclature
- **pyomo notation**:
  - $c_{i,j,h}$: Environmental cost matrix.
  - $b_{g,j}$: Intervention matrix.
  - $p_g$: Intervention flow vector.
- **article notation**:
  - $q_{h,e}$: Characterization factors.
  - $b_{e,j}$: Biosphere matrix.
  - No $p_g$ defined.

**Difference**: pyomo notation introduces additional matrices ($c_{i,j,h}$, $b_{g,j}$) and $p_g$, while article notation uses $q_{h,e}$ and $b_{e,j}$ directly.

---

#### Subset Usage
- **pyomo notation**:
  - Explicitly defines and uses subsets $PO_i$, $ECI_h$, $ECO_{i,h}$, and $IO_g$.
- **article notation**:
  - Uses general sets $i, j, e, h$ without subsets.

**Difference**: Subsets in pyomo notation provide additional granularity not present in article notation.

These are the key differences in variables, parameters, equations, and sets between the two formulations. Let me know if you'd like further clarification or a different format!